### Test code to update Argo Database from Erddap data

In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import datetime

server_url='http://akutan.pmel.noaa.gov:8080/erddap'
e = ERDDAP(server=server_url)

df_d = pd.read_csv(e.get_search_url(response='csv', search_for='drifter'))

drifter = df_d['Dataset ID'].values
print(drifter)

['2015_Argos_Drifters_NRT' '2016_Argos_Drifters_NRT'
 '2017_Argos_Drifters_NRT' '2018_Argos_Drifters_NRT'
 '2019_Argos_Drifters_NRT']


In [2]:
from requests.exceptions import HTTPError

dfs_2 = {}
for arfloat in drifter:
    print(arfloat)
    try:
        e = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        e.dataset_id=arfloat
    except HTTPError:
        print('Failed to generate url {}'.format(alamo))
        continue
    try:
        df = e.to_pandas(
            parse_dates=True,
            skiprows=(1,)  # units information can be dropped.
        ).dropna()
        
        df.sort_index(inplace=True)
        df.columns = [x[1].split()[0] for x in enumerate(df.columns)]

        dfs_2.update({arfloat: df})
    except:
        pass

2015_Argos_Drifters_NRT
2016_Argos_Drifters_NRT
2017_Argos_Drifters_NRT
2018_Argos_Drifters_NRT
2019_Argos_Drifters_NRT


In [3]:
for key,value in dfs_2.items():
    gb = value.groupby('trajectory_id')
    for name, group in gb:
        print(f"{name},{group.max()['time'].split('T')[0]}")
    

122533,2015-03-04
122537,2015-06-24
122542,2015-03-07
122843,2015-07-21
122845,2015-07-22
122846,2015-04-18
122847,2015-02-07
136859,2015-09-04
136860,2015-09-08
136861,2015-10-12
136862,2015-08-24
136863,2015-11-29
136864,2015-12-31
136865,2015-12-04
136866,2015-12-15
136867,2015-12-31
136868,2015-12-09
136869,2015-10-16
136870,2015-09-22
136871,2015-07-05
136873,2015-12-31
136874,2016-01-01
148276,2015-12-31
148277,2015-12-31
148278,2015-09-06
148279,2015-12-31
122537,2016-10-19
122540,2016-10-19
122541,2016-10-19
122542,2016-10-19
136860,2016-10-19
136861,2016-08-27
136863,2016-10-19
136864,2016-08-28
136865,2016-08-15
136866,2016-10-19
136867,2016-10-19
136868,2016-10-19
136869,2016-10-19
136872,2016-12-24
136873,2016-08-20
136874,2016-01-19
148276,2016-12-31
148277,2016-12-31
148279,2016-11-09
122537,2017-12-16
122540,2017-12-03
122541,2017-11-26
122542,2017-12-31
136860,2017-10-02
136863,2017-12-30
136866,2017-12-31
136867,2017-12-30
136868,2017-12-29
136869,2017-12-31
136872,201

In [4]:
# DB Stack
import mysql.connector


In [5]:
"""--------------------------------SQL Init----------------------------------------"""
class NumpyMySQLConverter(mysql.connector.conversion.MySQLConverter):
    """ A mysql.connector Converter that handles Numpy types """

    def _float32_to_mysql(self, value):
        if np.isnan(value):
            return None
        return float(value)

    def _float64_to_mysql(self, value):
        if np.isnan(value):
            return None
        return float(value)

    def _int32_to_mysql(self, value):
        if np.isnan(value):
            return None
        return int(value)

    def _int64_to_mysql(self, value):
        if np.isnan(value):
            return None
        return int(value)
    
def connect_to_DB(**kwargs):
    # Open database connection
    try:
        db = mysql.connector.connect(use_pure=True, **kwargs)
    except mysql.connector.Error as err:
        """
      if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
      elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
      else:
        print(err)
      """
        print("error - will robinson")

    db.set_converter_class(NumpyMySQLConverter)

    # prepare a cursor object using cursor() method
    cursor = db.cursor(dictionary=True)
    prepcursor = db.cursor(prepared=True)
    return (db, cursor)


def read_db(db, cursor, table, IsActive='Y'):
    sql = ("SELECT * from `{0}` WHERE `IsActive`= '{1}'").format(table, IsActive)

    result_dic = {}
    try:
        # Execute the SQL command
        cursor.execute(sql)
        # Get column names
        rowid = {}
        counter = 0
        for i in cursor.description:
            rowid[i[0]] = counter
            counter = counter +1 
        #print rowid
        # Fetch all the rows in a list of lists.
        results = cursor.fetchall()
        for row in results:
            result_dic[row['id']] ={keys: row[keys] for val, keys in enumerate(row.keys())} 
        return (result_dic)
    except:
        print("Error: unable to fecth data")

def update_db(db, cursor, table, ArgosID, date):
    sql = f"UPDATE {table} SET `LastKnownTransmission` = '{date}' WHERE ArgosNumber = '{ArgosID}'"
    print(sql)
    try:
        # Execute the SQL command
        cursor.execute(sql)

        db.commit()
    except mysql.connector.Error as error:
        print("Error: {}".format(error))
        db.rollback()

def close_DB(db):
    # disconnect from server
    db.close()
    
"""------------------------------------- Main -----------------------------------------"""

"""
# parse incoming command line options
parser = argparse.ArgumentParser(description='Create ActiveArgosIDs text file')
parser.add_argument('db_ini', metavar='db_ini', type=str, help='full path to db_config.pyini file')

args = parser.parse_args()

db_config = ConfigParserLocal.get_config(args.db_ini)
(db,cursor) = connect_to_DB(db_config['host'], db_config['user'], db_config['password'], db_config['database'])
"""

db_config = {"systems":{'pavlof':{"host":'pavlof',"port":3306}},
             "login":{"password":"e43mqS4fusEaGJLE","user":"pythonuser"},
             "database":{"database":"EcoFOCI_drifters"}}

host='pavlof'

(db, cursor) = connect_to_DB(
    host=db_config["systems"][host]["host"],
    user=db_config["login"]["user"],
    password=db_config["login"]["password"],
    database=db_config["database"]["database"],
    port=db_config["systems"][host]["port"],
)

table = 'drifter_ids'
data = read_db(db, cursor, table)

for key,value in dfs_2.items():
    gb = value.groupby('trajectory_id')
    for ArgosID, group in gb:
        print(f"{ArgosID},{group.max()['time'].split('T')[0]}")
        
        update_db(db,cursor,table,ArgosID,group.max()['time'].split('T')[0])
        
close_DB(db)


122533,2015-03-04
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-03-04' WHERE ArgosNumber = '122533'
122537,2015-06-24
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-06-24' WHERE ArgosNumber = '122537'
122542,2015-03-07
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-03-07' WHERE ArgosNumber = '122542'
122843,2015-07-21
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-07-21' WHERE ArgosNumber = '122843'
122845,2015-07-22
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-07-22' WHERE ArgosNumber = '122845'
122846,2015-04-18
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-04-18' WHERE ArgosNumber = '122846'
122847,2015-02-07
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-02-07' WHERE ArgosNumber = '122847'
136859,2015-09-04
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-09-04' WHERE ArgosNumber = '136859'
136860,2015-09-08
UPDATE drifter_ids SET `LastKnownTransmission` = '2015-09-08' WHERE ArgosNumber = '136860'
136861,2015-10-12
U